In [1]:
import os
import sys

# Edit it and put full path for crypto_crawler python project
CRYPTO_MODULE = "C:\Users\Dmitry\PycharmProjects\untitled"

if CRYPTO_MODULE not in sys.path:
    sys.path.append(CRYPTO_MODULE)
    
# for b in sys.path:
#    print b

In [2]:
from dao.db import init_pg_connection
pg_conn = init_pg_connection(_db_host="orders.cervsj06c8zw.us-west-1.rds.amazonaws.com",
                                 _db_port=5432, _db_name="crypto")

In [3]:
from utils.time_utils import get_now_seconds_utc

# Before stage of analysis we will try to update our db from info from exchange
should_fetch_data = False

# Epoch format !
time_end = get_now_seconds_utc()
time_start = time_end - 1 * 24 * 60 * 60

api_key_full_path = "C:\Users\Dmitry\PycharmProjects\untitled\secret_keys"

In [5]:
from analysis.data_load_for_profit_report import fetch_trades_history_to_db
from analysis.grouping_utils import group_trades_by_orders, group_by_pair_id
from analysis.data_preparation import prepare_data
from analysis.binance_order_by_trades import group_binance_trades_per_order
from utils.key_utils import load_keys
from collections import defaultdict, Counter


load_keys(api_key_full_path)

if should_fetch_data:
    fetch_trades_history_to_db(pg_conn, time_start)
    

# Load data from DB
orders, history_trades, binance_trades, binance_orders_at_bot, binance_orders_at_exchange = prepare_data(pg_conn, time_start)

# Group trades by orders
missing_orders, failed_orders, orders_with_trades = group_trades_by_orders(orders, history_trades, binance_orders_at_exchange)

# Group trades by orders for Binance
binance_trades_group_by_pair = group_by_pair_id(binance_trades)
binance_orders_with_trades = group_binance_trades_per_order(binance_orders_at_exchange, binance_trades_group_by_pair, binance_orders_at_bot)

orders_with_trades += binance_orders_with_trades

# Bucketing all that crap by pair_id
trades_to_order = defaultdict(list)
for order, trade_list in orders_with_trades:
    trades_to_order[order.pair_id].append( (order, trade_list))

In [7]:
from analysis.profit_report_analysis import compute_loss, compute_profit_by_pair, save_report

profit_by_pairs = Counter()
profit_by_pair_bitcoins = Counter()

for pair_id in trades_to_order:
    profit_by_pairs[pair_id], profit_by_pair_bitcoins[pair_id] = compute_profit_by_pair(pair_id, trades_to_order[pair_id])

overall_profit = sum(profit_by_pair_bitcoins.itervalues())

loss_by_pair, loss_by_pair_bitcoin = compute_loss(orders_with_trades)

save_report(time_start, time_end, overall_profit, profit_by_pairs, profit_by_pair_bitcoins,
            missing_orders, failed_orders, loss_by_pair, loss_by_pair_bitcoin)